[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mitiau/DNABERT-Z/blob/main/ZDNA-prediction.ipynb)

# Install dependecies and define helper functions

In [4]:
!pip install transformers
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.2 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
from google.colab import drive, files
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage

In [3]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res

def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

# Select model and parameters

In [4]:
model = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [5]:
if model == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
elif model == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
elif model == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
elif model == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'


In [6]:
!gdown $model_id
!gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
!gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
!gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
!gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0


!mkdir 6-new-12w-0
!mv pytorch_model.bin 6-new-12w-0/
!mv config.json 6-new-12w-0/
!mv special_tokens_map.json 6-new-12w-0/
!mv tokenizer_config.json 6-new-12w-0/
!mv vocab.txt 6-new-12w-0/

Downloading...
From (original): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx
From (redirected): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx&confirm=t&uuid=066b1e74-819f-42ae-931e-ccc3ed8484f7
To: /content/pytorch_model.bin
100% 354M/354M [00:02<00:00, 149MB/s] 
Downloading...
From: https://drive.google.com/uc?id=10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
To: /content/config.json
100% 634/634 [00:00<00:00, 3.64MB/s]
Downloading...
From: https://drive.google.com/uc?id=16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
To: /content/special_tokens_map.json
100% 112/112 [00:00<00:00, 455kB/s]
Downloading...
From: https://drive.google.com/uc?id=1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
To: /content/tokenizer_config.json
100% 40.0/40.0 [00:00<00:00, 188kB/s]
Downloading...
From: https://drive.google.com/uc?id=1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0
To: /content/vocab.txt
100% 28.7k/28.7k [00:00<00:00, 53.3MB/s]


In [7]:
tokenizer = BertTokenizer.from_pretrained('6-new-12w-0/')
model = BertForTokenClassification.from_pretrained('6-new-12w-0/')
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Upload fasta files for prediction

подгрузка организма

In [8]:
!wget -O 'genome.fna.gz' https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/190/715/GCF_000190715.1_v1.0/GCF_000190715.1_v1.0_genomic.fna.gz

--2025-06-16 11:19:05--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/190/715/GCF_000190715.1_v1.0/GCF_000190715.1_v1.0_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.12, 130.14.250.13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9942804 (9.5M) [application/x-gzip]
Saving to: ‘genome.fna.gz’

genome.fna.gz       100%[===================>]   9.48M  1.30MB/s    in 7.2s    

2025-06-16 11:19:15 (1.31 MB/s) - ‘genome.fna.gz’ saved [9942804/9942804]



In [9]:
!gzip -rd 'genome.fna.gz'

# Predict and save raw outputs

In [10]:
out = []
input_file ='genome.fna'
result_dict = {}

for seq_record in SeqIO.parse(input_file, 'fasta'):
    kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
    seq_pieces = split_seq(kmer_seq)
    print(seq_record.name)
    with torch.no_grad():
        preds = []
        for seq_piece in tqdm(seq_pieces):
            input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
            outputs = torch.softmax(model(input_ids.cuda().unsqueeze(0))[-1],axis = -1)[0,:,1]
            preds.append(outputs.cpu().numpy())
    result_dict[seq_record.name] = stitch_np_seq(preds)


    labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
    for label in range(1, max_label+1):
        candidate = np.where(labeled == label)[0]
        candidate_length = candidate.shape[0]
        if candidate_length>minimum_sequence_length:
            print(candidate[0], candidate[-1])
            out.append((seq_record.name, candidate[0], candidate[-1]))

NW_003519338.1


100%|██████████| 576/576 [00:25<00:00, 22.73it/s]


NW_003519339.1


100%|██████████| 543/543 [00:18<00:00, 29.33it/s]


NW_003519340.1


...

100%|██████████| 7/7 [00:00<00:00, 29.55it/s]


NW_003520135.1


100%|██████████| 7/7 [00:00<00:00, 28.87it/s]


812 829
NW_003520136.1


100%|██████████| 10/10 [00:00<00:00, 30.08it/s]


In [11]:
out

[('NW_003519340.1', np.int64(237983), np.int64(238001)),
 ('NW_003519363.1', np.int64(96677), np.int64(96690)),
 ('NW_003519441.1', np.int64(71900), np.int64(71913)),
 ('NW_003519555.1', np.int64(41978), np.int64(41996)),
 ('NW_003519931.1', np.int64(11906), np.int64(11918)),
 ('NW_003520041.1', np.int64(234), np.int64(251)),
 ('NW_003520041.1', np.int64(1269), np.int64(1282)),
 ('NW_003520110.1', np.int64(2288), np.int64(2305)),
 ('NW_003520119.1', np.int64(155), np.int64(172)),
 ('NW_003520135.1', np.int64(812), np.int64(829))]

In [12]:
with open('ZDNABERT.bed', 'a') as f:
    for record in out:
        f.write(record[0] + "\t" + str(record[1]) + "\t" + str(record[2]) + "\n")

In [13]:
!head ZDNABERT.bed

NW_003519340.1	237983	238001
NW_003519363.1	96677	96690
NW_003519441.1	71900	71913
NW_003519555.1	41978	41996
NW_003519931.1	11906	11918
NW_003520041.1	234	251
NW_003520041.1	1269	1282
NW_003520110.1	2288	2305
NW_003520119.1	155	172
NW_003520135.1	812	829


# Загрузим необходимые библиотеки

In [5]:
!apt-get install bedtools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 563 kB of archives.
After this operation, 1,548 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 bedtools amd64 2.30.0+dfsg-2ubuntu0.1 [563 kB]
Fetched 563 kB in 1s (530 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack .../bedtools_2.30.0+dfsg-2ubuntu0.1_amd64.deb ...
Unpacking bedtools (2.30.0+dfsg-2ubuntu0.1) ...
Setting up bedtools (2.30.0+dfsg-2ubuntu0.1) ...


# Скачивание аннотации и переведем аннотацию в необходимый формат и вытащим bed файлы промоторов/экзонов/межгенников/интронов/downstream

In [15]:
!wget -r -nH --cut-dirs=5 https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/190/715/GCF_000190715.1_v1.0/GCF_000190715.1_v1.0_genomic.gtf.gz

--2025-06-16 12:00:54--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/190/715/GCF_000190715.1_v1.0/GCF_000190715.1_v1.0_genomic.gtf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.31, 130.14.250.7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935297 (1.8M) [application/x-gzip]
Saving to: ‘715/GCF_000190715.1_v1.0/GCF_000190715.1_v1.0_genomic.gtf.gz’

715/GCF_000190715.1 100%[===================>]   1.84M  4.03MB/s    in 0.5s    

2025-06-16 12:00:55 (4.03 MB/s) - ‘715/GCF_000190715.1_v1.0/GCF_000190715.1_v1.0_genomic.gtf.gz’ saved [1935297/1935297]

FINISHED --2025-06-16 12:00:55--
Total wall clock time: 0.9s
Downloaded: 1 files, 1.8M in 0.5s (4.03 MB/s)


In [17]:
!gzip -rd /content/715/GCF_000190715.1_v1.0/GCF_000190715.1_v1.0_genomic.gtf.gz

# Сделаем все для zdnabert

In [6]:
!bedtools intersect -a ZDNABERT.bed -b exons.bed -wa > ZDNABERT_in_exons.bed

In [7]:
!bedtools intersect -a ZDNABERT.bed -b introns.bed -wa > ZDNABERT_in_introns.bed

In [9]:
!bedtools intersect -a ZDNABERT.bed -b promoters.bed -wa > ZDNABERT_in_promoters.bed

In [8]:
!bedtools intersect -a ZDNABERT.bed -b intergenic.bed -wa > ZDNABERT_in_intergenic.bed

In [10]:
!bedtools intersect -a ZDNABERT.bed -b downstream.bed -wa > ZDNABERT_in_downstream.bed

In [23]:
!cat ZDNABERT.bed | wc -l

10


In [3]:
!cat ZDNABERT.bed

NW_003519340.1	237983	238001
NW_003519363.1	96677	96690
NW_003519441.1	71900	71913
NW_003519555.1	41978	41996
NW_003519931.1	11906	11918
NW_003520041.1	234	251
NW_003520041.1	1269	1282
NW_003520110.1	2288	2305
NW_003520119.1	155	172
NW_003520135.1	812	829


In [11]:
!cat ZDNABERT_in_exons.bed | wc -l

3


In [12]:
!cat ZDNABERT_in_exons.bed

NW_003519340.1	237983	238001
NW_003519555.1	41978	41996
NW_003519931.1	11906	11918


In [25]:
!cat ZDNABERT_in_introns.bed | wc -l

0


In [26]:
!cat ZDNABERT_in_promoters.bed | wc -l

0


In [27]:
!cat ZDNABERT_in_intergenic.bed | wc -l

7


In [13]:
!cat ZDNABERT_in_intergenic.bed

NW_003519363.1	96677	96690
NW_003519441.1	71900	71913
NW_003520041.1	234	251
NW_003520041.1	1269	1282
NW_003520110.1	2288	2305
NW_003520119.1	155	172
NW_003520135.1	812	829


In [28]:
!cat ZDNABERT_in_downstream.bed | wc -l

1


Участки

In [16]:
!bedtools intersect -a exons.bed -b ZDNABERT.bed -wa > exons_ZDNABERT.bed

In [17]:
!bedtools intersect -a introns.bed -b ZDNABERT.bed -wa > introns_ZDNABERT.bed

In [18]:
!bedtools intersect -a promoters.bed -b ZDNABERT.bed -wa > promoters_ZDNABERT.bed

In [19]:
!bedtools intersect -a intergenic.bed -b ZDNABERT.bed -wa > intergenic_ZDNABERT.bed

In [20]:
!bedtools intersect -a downstream.bed -b ZDNABERT.bed -wa > downstream_ZDNABERT.bed

In [14]:
!cat introns.bed | wc -l

18744


In [21]:
!cat introns_ZDNABERT.bed | wc -l

0


In [22]:
!cat exons.bed | wc -l

31143


In [23]:
!cat exons_ZDNABERT.bed | wc -l

3


In [24]:
!cat exons_ZDNABERT.bed

NW_003519340.1	237627	238911	exon	"DICPUDRAFT_74136";	-
NW_003519555.1	40981	42370	exon	"DICPUDRAFT_81116";	+
NW_003519931.1	11701	12506	exon	"DICPUDRAFT_51633";	+


In [26]:
3/31143

9.632983334938831e-05

In [27]:
!cat promoters.bed | wc -l

12331


In [28]:
!cat promoters_ZDNABERT.bed | wc -l

0


In [29]:
!cat intergenic.bed | wc -l

13038


In [30]:
!cat intergenic_ZDNABERT.bed | wc -l

7


In [31]:
!cat intergenic_ZDNABERT.bed

NW_003519363.1	96223	98485
NW_003519441.1	70651	72000
NW_003520041.1	0	5795
NW_003520041.1	0	5795
NW_003520110.1	0	3987
NW_003520119.1	0	3813
NW_003520135.1	0	3170


In [32]:
7/13038

0.0005368921613744439

In [33]:
!cat downstream.bed | wc -l

12385


In [34]:
!cat downstream_ZDNABERT.bed | wc -l

1


In [35]:
1/12385

8.074283407347598e-05